<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>  

# Práctica 1 - EDOS (parte 3: data augmentation)

SemEval 2023 - Task 10 - Explainable Detection of Online Sexism (EDOS)

En este notebook, usaremos técnicas para data augmentation para aumentar el conjunto de entrenamiento de la tarea EDOS.


Recuerda que el conjunto de datos para la tarea EDOS (https://github.com/rewire-online/edos).

NOTA PARA PODER EJECUTAR ESTE NOTEBOOK:

1) Para poder ejercutar correctamente este notebook, deberás abrirlo en tu Google Drive (por ejemplo, en la carpeta 'Colab Notebooks').

2) Además, debes guardar el dataset en tu Google Drive, dentro de carpeta 'Colab Notebooks/data/edos/'.

En el tema 6, estudiamos distintas librerías y enfoques para la generación de textos automáticos como 'back translation', la librería NLPAug o la librería textaugmenter.

En este notebook, nos limitaremos a usar la librería textaugmenter, pero te recomendamos que pruebes con las otras técnicas y veas qué efecto tienen sobre los resultados de los transformers aplicados a EDOS.


# Instalar librerías


In [1]:
!pip install -q transformers[torch] datasets textaugment nlpaug


## Cargar el dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/data/edos/')

import pandas as pd
df = pd.read_csv("edos_labelled.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-16993,"Then, she's a keeper. 😉",not sexist,none,none,train
2,sexism2022_english-13149,This is like the Metallica video where the poo...,not sexist,none,none,train
3,sexism2022_english-13021,woman?,not sexist,none,none,train
4,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev


Las técnicas de DA únicamente deben ser aplicadas sobre los ejemplos del conjunto de entrenamiento, para generar nuevos textos.
Nunca deben ser aplicadas sobre el conjunto de validación o test.

Por tanto, en este notebook, únicamente cargaremos el conjunto de training:

In [3]:
from datasets import DatasetDict, Dataset
df_train = df[df['split']=='train']
dataset = Dataset.from_pandas(df_train)
dataset

Dataset({
    features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector', 'split', '__index_level_0__'],
    num_rows: 14000
})

Vamos a aplicar las librerías **textaugment** y **NLPAug** que hemos estudiado durante el curso.

Ambas librerías ofrecen distintas técnicas para generar nuevos textos sintéticos, pero en este tutorial vamos a utilizar las técnicas basadas en reemplazo de sinónimos.

En el caso de **textaugment**, esta librería usa WordNet para buscar los sinónimos. Por ese motivo, necesitmaos instalar la librería nltk, que incluye el recurso WordNet.


In [4]:
import nltk
# necesitamos instalar algunos modulos de nltk para poder usar el método synonym_replacement de EDA
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Ahora ya podeos definir una función **generate_eda** que aplique el método **synonym_replacement** sobre cada texto de entrada para generar un nuevo texto. En concreto, la función **generate_eda** recibe un lote de instancias, y por cada instancia del dataset, toma su texto, genera uno nuevo usando el método anteriormente citado, sustituyendo el original por el nuevo. Finalmente, la función devuelve el conjunto de instancias actualizado con los nuevos textos. El resto de campos se mantienen.

 La función **generate_eda** es aplicada sobre todo el conjunto de entrenamiento:

In [17]:
from textaugment import EDA
t = EDA()


def generate_eda(examples):
    """recibe un lote de instancias. Por cada instancia, toma el texto y genera uno nuevo.
    Entonces reemplaza el texto original con el nuevo. Finalente, devuelve el lote"""
    new_texts = []
    for text in examples['text']:
        new_text = t.synonym_replacement(text)
        new_texts.append(new_text)
    examples['text'] = new_texts
    return examples

new_training_data_eda = dataset.map(generate_eda, batched = True)
new_training_data_eda

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Dataset({
    features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector', 'split', '__index_level_0__'],
    num_rows: 14000
})

El dataset new_training_data contiene las mismas instancias con los mismos campos, excepto el campo texto, que ahora contiene un nuevo texto sintético generado a partir del texto original.

Vamos a comparar algunas instancias originales con las nuevas generadas

In [19]:
import random
index = random.randint(0,dataset.num_rows)
print("Instancia original: ", dataset[index])
print("Instancia generada: ", new_training_data_eda[index])


Instancia original:  {'rewire_id': 'sexism2022_english-17040', 'text': "The left has a disturbing obsession with infanticide, rape, and women's sexual organs. It's the stuff of severely unwell individuals. And I'm surprised none of them tried to hit her in the stomach, TBH.", 'label_sexist': 'not sexist', 'label_category': 'none', 'label_vector': 'none', 'split': 'train', '__index_level_0__': 7589}
Instancia generada:  {'rewire_id': 'sexism2022_english-17040', 'text': "The left has a disturbing obsession with infanticide, rape, and women's sexual organs. It's the stuff of severely indisposed individuals. And I'm surprised none of them tried to hit her in the stomach, TBH.", 'label_sexist': 'not sexist', 'label_category': 'none', 'label_vector': 'none', 'split': 'train', '__index_level_0__': 7589}


Vamos a grabar en un fichero estas instancias, que luego usaremos para entrenar el modelo junto con el training original

In [38]:
path = "/content/drive/My Drive/Colab Notebooks/data/edos/"
new_training_data_eda.to_csv(path + "edos_aug_eda.csv", index = None)
print('fichero grabado')

Creating CSV from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

fichero grabado


A continuación utilizaremos la librería NLPAug. Como hemos visto durante el curso, la librería proporciona distintas técnicas pero en este cuaderno, trabajaremos con la clase ContextualWordEmbsAug, que permite cargar un modelo de lenguaje como BERT para generar nuevos textos.

El proceso puede durar bastantes minutos. Para probar, simplemente lo vamos a utilizar sobre una pequeña muestra del conjunto de entrenamiento

In [36]:
sample = dataset.shuffle(seed=42).select(range(100))
# sample[0]


In [41]:
import nlpaug.augmenter.word as naw     # modelo de lenguaje (como BERT) para generar nuevas instancias
nlpaug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

def generate_nlpaug(examples):
    """recibe un lote de instancias. Por cada instancia, toma el texto y genera uno nuevo.
    Entonces reemplaza el texto original con el nuevo. Finalente, devuelve el lote"""
    new_texts = []
    for text in examples['text']:
        new_text = nlpaug.augment(text)
        # augment devuelve una lista de textos, tomamos el primer texto
        new_texts.append(new_text[0])
    examples['text'] = new_texts
    return examples


new_training_data_nlpaug = sample.map(generate_nlpaug, batched = True)
new_training_data_nlpaug

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector', 'split', '__index_level_0__'],
    num_rows: 100
})

Comparamos algunas instancias:

In [44]:
import random
index = random.randint(0,sample.num_rows)
print("Instancia original: ", sample[index])
print("Instancia generada: ", new_training_data_nlpaug[index])

Instancia original:  {'rewire_id': 'sexism2022_english-16173', 'text': "I read posts on here only occasionally and contribute even less. I hadn't noticed that many problematic male posts but i could see that if men come on here too often and drown out women it kind of defeats the whole point of the sub.", 'label_sexist': 'not sexist', 'label_category': 'none', 'label_vector': 'none', 'split': 'train', '__index_level_0__': 16615}
Instancia generada:  {'rewire_id': 'sexism2022_english-16173', 'text': "i could read posts on under here too only occasionally and contribute with even little less. i hadn't noticed that many problematic male posts earlier but i only could see that if old men come down on here too often and drown me out women it kind of defeats the whole point of the sub.", 'label_sexist': 'not sexist', 'label_category': 'none', 'label_vector': 'none', 'split': 'train', '__index_level_0__': 16615}


Vamos a aplicar también el método back translation implementado en la libraría textaugment. Como idioma intermedio usaremos el español, pero podríamos probar con cualquier otro. El proceso puede tardar bastantes minutos, y además después de un número de traducciones, el acceso al servicio de traducción puede estar restringido. Por ese motivo, únicamente lo vamos a aplicar sobre una pequeña muestra en la siguiente celda:

In [ ]:
from textaugment import Translate
SRC = "en" # idioma de los textos
TO = "es" # idioma que se utiliza para traducir como paso intermedio

t = Translate(src=SRC, to=TO)

def generate_backt(examples):
    # reemplazamos el texto original con el texto generado usando back translation
    new_texts = []
    for text in examples['text']:
        new_text = t.augment(text)
        new_texts.append(new_text)
    examples['text'] = new_texts
    return examples


new_training_data_backt = sample.map(generate_backt, batched = True)
new_training_data_backt

In [ ]:
import random
index = random.randint(0,sample.num_rows)
print("Instancia original: ", sample[index])
print("Instancia generada: ", new_training_data_backt[index])
